In [6]:
%pip install dotenv

  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)

[notice] A new release of pip is available: 24.1.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from newsplease import NewsPlease


In [7]:
import pandas as pd

url = "https://api.gdeltproject.org/api/v2/doc/doc"
params = {
    "query": "climate",   # search term
    "mode": "ArtList",           # return article list
    "maxrecords": "250",         # max results (1–250)
    "format": "CSV",             # output format
    "sort": "DateDesc"           # newest first
}

df = pd.read_csv(f"{url}?query={params['query']}&mode={params['mode']}&maxrecords={params['maxrecords']}&format={params['format']}&sort={params['sort']}")
print(df.head())

                                                 URL MobileURL  \
0  https://www.cqnews.net/1/detail/14317780277842...       NaN   
1           https://n.yam.com/Article/20251026608541       NaN   
2  https://www.lancashiretelegraph.co.uk/news/nat...       NaN   
3  https://www.elliberal.com.ar/nota/62259/2025/1...       NaN   
4  https://www.yahoo.com/news/articles/war-climat...       NaN   

                  Date                                              Title  
0  2025-10-25 16:45:00                大量燕子滞留 ， 不少冻死街头 ！ 它们为啥来不及往南飞 ？- 华龙网  
1  2025-10-25 16:45:00               一對一單打王籃球賽在大稻埕河濱公園舉行 52歲vs . 17歲的老少對決  
2  2025-10-25 16:45:00  Catherine Connolly needed Sinn Fein support – ...  
3  2025-10-25 16:45:00  Juan Domingo Perón , el hombre de las tres pre...  
4  2025-10-25 16:45:00  War and Climate Change Are Redrawing Global Tr...  


In [3]:
from google.cloud import bigquery
import dotenv
dotenv.load_dotenv()
import os
client = bigquery.Client(project = os.getenv("GOOGLE_CLOUD_PROJECT"))

query = """
SELECT
  DATE(PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING))) AS date,
  Actor1CountryCode,
  Actor2CountryCode,
  GoldsteinScale,
  AvgTone,
  EventRootCode
FROM
  `gdelt-bq.gdeltv2.events`
WHERE
  DATE(PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING))) > '2025-01-01'
  AND Actor1CountryCode = 'USA'
LIMIT 1000
"""

df = client.query(query).to_dataframe()
print(df.head())

         date Actor1CountryCode Actor2CountryCode  GoldsteinScale   AvgTone  \
0  2025-10-25               USA               SEN             1.9  2.960526   
1  2025-10-25               USA              None             2.8 -1.762632   
2  2025-10-25               USA              None           -10.0  2.433090   
3  2025-10-25               USA              None             4.0  2.033898   
4  2025-10-25               USA              None            -5.0 -0.078740   

  EventRootCode  
0            04  
1            04  
2            18  
3            03  
4            12  


In [4]:
df

,date,Actor1CountryCode,Actor2CountryCode,GoldsteinScale,AvgTone,EventRootCode
0,2025-10-25,USA,SEN,1.9,2.960526,04
1,2025-10-25,USA,None,2.8,-1.762632,04
2,2025-10-25,USA,None,-10.0,2.433090,18
3,2025-10-25,USA,None,4.0,2.033898,03
4,2025-10-25,USA,None,-5.0,-0.078740,12
...,...,...,...,...,...,...
995,2025-09-22,USA,None,3.0,-4.657728,02
996,2025-09-22,USA,None,-5.0,-4.657728,17
997,2025-10-22,USA,None,-2.0,-0.920952,11
998,2025-10-23,USA,None,7.0,3.429796,08


In [5]:
def retrieve(query):
    client = bigquery.Client(project = os.getenv("GOOGLE_CLOUD_PROJECT"))
    return client.query(query).to_dataframe()

In [8]:
query = """
        SELECT
        PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING)) AS date,
        COUNT(*) AS total_events
        FROM
        `gdelt-bq.gdeltv2.events`
        WHERE
        SQLDATE >= 20251001
        GROUP BY date
        ORDER BY date DESC
        LIMIT 10
        """
retrieve(query)

,date,total_events
0,2025-10-25,121961
1,2025-10-24,200734
2,2025-10-23,209794
3,2025-10-22,213931
4,2025-10-21,209495
5,2025-10-20,182441
6,2025-10-19,115436
7,2025-10-18,133124
8,2025-10-17,210162
9,2025-10-16,215014


In [11]:
query = """
SELECT
  PARSE_DATE('%Y%m%d', CAST(e.SQLDATE AS STRING)) AS event_date,
  e.Actor1CountryCode,
  e.Actor2CountryCode,
  e.AvgTone AS event_tone,
  g.tone AS article_tone,
  g.themes
FROM
  `gdelt-bq.gdeltv2.events` AS e
JOIN
  `gdelt-bq.gdeltv2.gkg` AS g
ON
  PARSE_DATE('%Y%m%d', CAST(e.SQLDATE AS STRING)) = PARSE_DATE('%Y%m%d', CAST(g.DATE AS STRING))
WHERE
  e.Actor1CountryCode = 'USA'
  AND g.themes LIKE '%ELECTION%'
  AND e.SQLDATE BETWEEN 20250901 AND 20251024
LIMIT 100
        """
retrieve(query)

BadRequest: 400 Name tone not found inside g at [7:5]; reason: invalidQuery, location: query, message: Name tone not found inside g at [7:5]

Location: US
Job ID: 05dc92b9-9c7b-4ab8-b4b3-f8f1426f4c3d


In [13]:
query = """
SELECT
  ActionGeo_CountryCode AS country,
  SUM(NumMentions) AS total_mentions
FROM `gdelt-bq.gdeltv2.events`
WHERE SQLDATE >= 20241001
GROUP BY country
ORDER BY total_mentions DESC
LIMIT 10;
"""
retrieve(query)

,country,total_mentions
0,US,72565500
1,None,21570605
2,RS,18869380
3,IS,18270541
4,UP,15844834
5,UK,14467912
6,IN,14326686
7,CH,8916313
8,IT,7819273
9,MX,7248153


In [36]:
query = """SELECT
  DocumentIdentifier AS article_url,
  SourceCommonName AS source,
  V2Themes,
  V2Tone
FROM `gdelt-bq.gdeltv2.gkg`
WHERE DATE >= 20251024
LIMIT 30;"""
df =retrieve(query)

In [37]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(df['article_url'].head(10))
df['article_url'].head()

0       https://www.0223.com.ar/nota/2024-7-31-7-38-0-tras-un-reclamo-en-la-empresa-servisa-y-sus-trabajadores-van-al-ministerio-de-trabajo
1                      https://www.0223.com.ar/nota/2024-7-31-10-18-0-denuncian-a-la-empresa-meta-por-el-uso-de-datos-privados-en-argentina
2                                   https://www.0223.com.ar/nota/2024-7-31-9-37-0-boca-obligado-a-ganar-y-mejorar-su-imagen-en-la-bombonera
3                 https://www.0223.com.ar/nota/2024-7-31-9-28-0-gallardo-ajusta-detalles-de-su-regreso-y-seria-presentado-la-proxima-semana
4    https://www.0223.com.ar/nota/2024-7-31-9-25-0-kicillof-sobre-la-planta-de-gnl-el-directorio-de-ypf-tomo-una-decision-dictada-por-milei
5                                 https://www.0223.com.ar/nota/2024-7-31-9-5-0-diana-mondino-esta-en-la-oea-por-las-elecciones-en-venezuela
6            https://www.0223.com.ar/nota/2024-7-31-7-36-0-familiares-de-un-joven-que-aparecio-muerto-piden-testigos-y-camaras-de-seguridad
7                   

0       https://www.0223.com.ar/nota/2024-7-31-7-38-0-tras-un-reclamo-en-la-empresa-servisa-y-sus-trabajadores-van-al-ministerio-de-trabajo
1                      https://www.0223.com.ar/nota/2024-7-31-10-18-0-denuncian-a-la-empresa-meta-por-el-uso-de-datos-privados-en-argentina
2                                   https://www.0223.com.ar/nota/2024-7-31-9-37-0-boca-obligado-a-ganar-y-mejorar-su-imagen-en-la-bombonera
3                 https://www.0223.com.ar/nota/2024-7-31-9-28-0-gallardo-ajusta-detalles-de-su-regreso-y-seria-presentado-la-proxima-semana
4    https://www.0223.com.ar/nota/2024-7-31-9-25-0-kicillof-sobre-la-planta-de-gnl-el-directorio-de-ypf-tomo-una-decision-dictada-por-milei
Name: article_url, dtype: object

In [59]:
query = """
#standardSQL
-- Corrected Query: retrieves GDELT 2.0 Knowledge Graph (GKG) records added within the last 24 hours.
-- We use the 'DATE' field which holds the YYYYMMDDHHMMSS timestamp in the partitioned table.
SELECT
  -- The unique URL of the source article
  DocumentIdentifier AS ArticleURL,
  -- GDELT's standard date/time stamp (YYYYMMDDHHMMSS)
  DATE,
  -- The main themes/keywords extracted from the article
  V2Themes AS Themes,
  -- Major locations mentioned
  V2Locations AS Locations,
  -- Major people mentioned
  V2Persons AS Persons,
  -- Major organizations mentioned
  V2Organizations AS Organizations
FROM
  `gdelt-bq.gdeltv2.gkg_partitioned`
WHERE
  -- Convert the integer DATE (YYYYMMDDHHMMSS) into a BigQuery TIMESTAMP
  -- and check if it is greater than or equal to the current time minus 24 hours (1 day).
  PARSE_TIMESTAMP('%Y%m%d%H%M%S', CAST(DATE AS STRING))
    >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 24 HOUR)
ORDER BY
  DATE DESC
LIMIT
  1000

"""
df = retrieve(query)
df

ArticleURL  \
0                                                                                                                                                     https://nypost.com/2025/10/25/betting/ravens-bettors-burned-by-lamar-jackson-injury-confusion/   
1    https://www.cairnspost.com.au/news/national/back-australia/why-aussie-family-looks-for-australian-made-items-as-foreignmade-groceries-sit-on-shelves/news-story/12965a0bb0ebea33527cf6c3cf5c0d58?nk=eee8b3166cf066b302a93c91315664ca-1761428813   
2                                                                                                                                                                https://wmscradio.com/a-grand-slam-bull-durham-the-musical-at-paper-mill-playhouse/   
3                                                                                                                          https://www.braintreeandwithamtimes.co.uk/news/national/25572094.kamala-harris-leaves-door-open-2028-us-presidential-run/   
4                                                                                                                                                    https://www.scoop.co.nz/stories/HL2510/S00060/health-minister-unethical-over-medical-ethics.htm   
..                                                                                                                                                                                                                                               ...   
995                                              https://timesofindia.indiatimes.com/technology/social/chhath-puja-2025-captions-for-instagram-and-facebook-beautiful-and-simple-ideas-to-share-the-spirit-of-the-festival/articleshow/124818376.cms   
996                                                                                                     https://www.kreiszeitung.de/boulevard/schwerer-reitunfall-tante-von-prinz-william-und-prinz-harry-wochenlang-im-krankenhaus-zr-94005446.html   
997                                                                                                                                 https://nypost.com/2025/10/25/lifestyle/furloughed-irs-lawyer-achieves-childhood-dream-of-operating-hotdog-cart/   
998                                                                                                                           https://timesofindia.indiatimes.com/india/judges-mustnt-yield-to-lawyers-pressure-gujarat-hc/articleshow/124818511.cms   
999                                                                                                  https://www.djournal.com/opinion/columnists/dont-let-our-kids-pay-the-price-for-school-choice/article_fe6ab233-271c-4a3c-8438-0a0698c0b02d.html   

               DATE  \
0    20251025220000   
1    20251025220000   
2    20251025220000   
3    20251025220000   
4    20251025220000   
..              ...   
995  20251025214500   
996  20251025214500   
997  20251025214500   
998  20251025214500   
999  20251025214500   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [49]:
df.set_index("DATE").sort_index()

GKGRECORDID  SourceCollectionIdentifier  \
DATE                                                              
20191207154500   20191207154500-925                           1   
20191207164500   20191207164500-586                           1   
20191207171500   20191207171500-645                           1   
20191207173000  20191207173000-1314                           1   
20191207174500   20191207174500-153                           1   
...                             ...                         ...   
20200119001500   20200119001500-921                           1   
20200119011500   20200119011500-209                           1   
20200119020000   20200119020000-893                           1   
20200119043000   20200119043000-806                           1   
20200119071500   20200119071500-683                           1   

               SourceCommonName  \
DATE                              
20191207154500         kmbc.com   
20191207164500          msn.com   
20191207171500          ajc.com   
20191207173000         ktuu.com   
20191207174500       news24.com   
...                         ...   
20200119001500       iheart.com   
20200119011500         whio.com   
20200119020000       nasdaq.com   
20200119043000        bbc.co.uk   
20200119071500       livelaw.in   

                                                                                                                                                         DocumentIdentifier  \
DATE                                                                                                                                                                          
20191207154500                                  https://www.kmbc.com/article/former-police-officer-accused-of-putting-dog-leash-around-stepsons-neck-choking-him-1/30157069   
20191207164500  https://www.msn.com/en-us/travel/article/families-of-fallen-military-heroes-get-festive-send-off-as-e2-80-98snowball-express-e2-80-99-leaves-lax/ar-BBXU39W   
20191207171500                                                      https://www.ajc.com/business/these-atlanta-starbucks-will-get-major-renovations/DYieNmFlfsvVMUlfHqM67L/   
20191207173000                                                   https://www.ktuu.com/content/news/Ex-police-officer-accused-of-choking-child-with-dog-leash-565925021.html   
20191207174500                                                                     https://www.news24.com/SouthAfrica/News/two-arrested-in-home-affairs-fraud-bust-20191207   
...                                                                                                                                                                     ...   
20200119001500                                https://mix967online.iheart.com/content/2020-01-18-prince-harry-meghan-markle-to-lose-hrh-titles-as-queen-announces-new-deal/   
20200119011500                       https://www.whio.com/news/local/watch-hamilton-girl-battling-cancer-surprised-with-white-horse-dressed-unicorn/sN6qqsBnLv9Xwv4YLD8CDJ/   
20200119020000                                                                                         https://www.nasdaq.com/articles/can-dave-busters-be-saved-2020-01-18   
20200119043000                                                                   https://www.bbc.co.uk/news/av/world-asia-india-51148001/the-tiny-recipe-to-youtube-success   
20200119071500                            https://www.livelaw.in/news-updates/advocates-can-be-appointed-as-receivers-us-141a-of-sarfaesi-act-delhi-hc-read-judgment-151796   

                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [57]:
from gdeltdoc import GdeltDoc, Filters

# Correct timespan syntax: integer + unit (no space)
f = Filters(
    keyword="climate change",
    timespan="10m"  # <- "day" not "days", no space
)

gd = GdeltDoc()

articles = gd.article_search(f)

for a in articles:
    print(a)

url
url_mobile
title
seendate
socialimage
domain
language
sourcecountry


In [51]:
%pip install gdeltdoc

Note: you may need to restart the kernel to use updated packages.


In [61]:
query = """SELECT
  -- The unique URL of the source article
  DocumentIdentifier AS ArticleURL,
  -- GDELT's standard date/time stamp (YYYYMMDDHHMMSS)
  DATE,
  -- The people mentioned in the article (where we expect to find Modi)
  V2Persons AS MentionedPersons,
  -- The main themes/keywords extracted from the article
  V2Themes AS Themes,
  -- Major locations mentioned
  V2Locations AS Locations,
  -- Major organizations mentioned
  V2Organizations AS Organizations
FROM
  `gdelt-bq.gdeltv2.gkg_partitioned`
WHERE
  -- 1. Filter by time: check if the date is within the last 48 hours
  PARSE_TIMESTAMP('%Y%m%d%H%M%S', CAST(DATE AS STRING))
    >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 48 HOUR)
  -- 2. Filter by content: look for 'modi' in the list of mentioned people (case-insensitive)
  AND LOWER(V2Persons) LIKE '%modi%'
ORDER BY
  DATE DESC
LIMIT
  1000"""
df = retrieve(query)

df

,ArticleURL,DATE,MentionedPersons,Themes,Locations,Organizations
0,https://www.dnaindia.com/india/report-bihar-election-2025-news-updates-union-home-minister-amit-shah-slams-opposition-mahagathbandhan-alliance-says-i-pray-to-chhath-maiya-that-the-state-remains-free-from-jungle-raj-3185396,20251025220000,"Chhath Maiya,597;Chhath Maiya,831;Narendra Modi,877;Narendra Modi,2813;Nitish Kumar,2222;Amit Shah,29;Amit Shah,304;Amit Shah,2364;Amit Shah,3951;Amit Shah,4536;Amit Shah,5098;Amit Shah,5505;Bhumi Pujan,2880;Mata Sita Charan Temple,3062;Ram Lalla,2650;Pran Prathishtha,2932;Chhathi Maiya,3977;Rashtriya Janata Dal,5955;Rahul Baba,5263;Lalu Yadav,2738;Mamata Banerjee,2723;Ashok Gehlot,3606;Ram Temple,2537","TAX_POLITICAL_PARTY_RASHTRIYA_JANATA_DAL,5955;LEADER,863;LEADER,2835;TAX_FNCACT_PRIME_MINISTER,863;TAX_FNCACT_PRIME_MINISTER,2835;TAX_FNCACT_MINISTER,19;TAX_FNCACT_MINISTER,863;TAX_FNCACT_MINISTER,2835;TAKE_OFFICE,1448;TAX_TERROR_GROUP_COMMUNIST_PARTY_OF_INDIA,6010;TAX_TERROR_GROUP_COMMUNIST_PARTY_OF_INDIA,6065;TAX_TERROR_GROUP_COMMUNIST_PARTY_OF_INDIA,6100;TAX_POLITICAL_PARTY_COMMUNIST_PARTY_OF_INDIA,6010;TAX_POLITICAL_PARTY_COMMUNIST_PARTY_OF_INDIA,6065;TAX_POLITICAL_PARTY_COMMUNIST_PARTY_OF_INDIA,6100;TAX_ETHNICITY_HINDUSTANI,5860;TAX_WORLDLANGUAGES_HINDUSTANI,5860;TAX_POLITICAL_PARTY_NATIONAL_DEMOCRATIC_ALLIANCE,3786;EPU_CATS_MIGRATION_FEAR_FEAR,1165;EPU_CATS_MIGRATION_FEAR_FEAR,1784;EPU_CATS_NATIONAL_SECURITY,1165;EPU_CATS_NATIONAL_SECURITY,1784;MANMADE_DISASTER_IMPLIED,1543;CORRUPTION,3443;LEGISLATION,659;LEGISLATION,2053;LEGISLATION,2115;EPU_POLICY_LAW,659;EPU_POLICY_LAW,2053;EPU_POLICY_LAW,2115;ELECTION,109;ELECTION,1963;ELECTION,2184;ELECTION,2321;ELECTION,3706;ELECTION,4041;ELECTION,4382;ELECTION,4878;ELECTION,5563;RELIGION,4429;CRISISLEX_T11_UPDATESSYMPATHY,581;CRISISLEX_T11_UPDATESSYMPATHY,815;EPU_POLICY_GOVERNMENT,1002;EPU_POLICY_GOVERNMENT,1299;EPU_POLICY_GOVERNMENT,2086;EPU_POLICY_GOVERNMENT,2286;EPU_POLICY_GOVERNMENT,5183;EPU_POLICY_GOVERNMENT,5398;UNEMPLOYMENT,4136;TAX_FNCACT_UNEMPLOYED,4136;UNGP_JOB_OPPORTUNITIES_EMPLOYMENT,4136;USPEC_UNCERTAINTY1,2755;ARMEDCONFLICT,1550;KILL,1868;CRISISLEX_T03_DEAD,1868;TAX_FNCACT_COMMUNIST,5995;TAX_FNCACT_COMMUNIST,6050;TAX_FNCACT_COMMUNIST,6085;TAX_FNCACT_LEADER,1231;TAX_FNCACT_LEADER,3551;TAX_FNCACT_LEADER,3636;TAX_FNCACT_LEADER,3904;DEMOCRACY,3777;SOC_POINTSOFINTEREST_AIRPORT,3362;WB_135_TRANSPORT,3362;WB_1803_TRANSPORT_INFRASTRUCTURE,3362;WB_1804_AIRPORTS,3362;GENERAL_GOVERNMENT,922;GENERAL_GOVERNMENT,2694;GENERAL_GOVERNMENT,3544;GENERAL_GOVERNMENT,3897;GENERAL_GOVERNMENT,4553;GENERAL_GOVERNMENT,4651;GENERAL_GOVERNMENT,4777;GENERAL_GOVERNMENT,5122;GENERAL_GOVERNMENT,5979;USPEC_POLITICS_GENERAL1,922;USPEC_POLITICS_GENERAL1,2694;USPEC_POLITICS_GENERAL1,3544;USPEC_POLITICS_GENERAL1,3897;USPEC_POLITICS_GENERAL1,4553;USPEC_POLITICS_GENERAL1,4651;USPEC_POLITICS_GENERAL1,4777;USPEC_POLITICS_GENERAL1,5122;USPEC_POLITICS_GENERAL1,5979;EPU_POLICY_CONGRESS,922;EPU_POLICY_CONGRESS,2694;EPU_POLICY_CONGRESS,3544;EPU_POLICY_CONGRESS,3897;EPU_POLICY_CONGRESS,4553;EPU_POLICY_CONGRESS,4651;EPU_POLICY_CONGRESS,4777;EPU_POLICY_CONGRESS,5122;EPU_POLICY_CONGRESS,5979;TAX_POLITICAL_PARTY_COMMUNIST_PARTY,6001;TAX_POLITICAL_PARTY_COMMUNIST_PARTY,6056;TAX_POLITICAL_PARTY_COMMUNIST_PARTY,6091;TAX_TERROR_GROUP_COMMUNIST_PARTY_OF_INDIA_MARXIST,6018;TAX_TERROR_GROUP_COMMUNIST_PARTY_OF_INDIA_MARXIST,6108;TERROR,1173;TAX_TERROR_GROUP_BHARATIYA_JANATA_PARTY,5791;TAX_POLITICAL_PARTY_BHARATIYA_JANATA_PARTY,5791;ALLIANCE,3786;ALLIANCE,4230;ALLIANCE,5735;CRISISLEX_CRISISLEXREC,1791;CRISISLEX_CRISISLEXREC,3486;TAX_POLITICAL_PARTY_JANATA_DAL,5808;TAX_POLITICAL_PARTY_JANATA_DAL,5955;","4#Bihar, Bihar, India#IN#IN34#17615#25.1833#85.5167#-2091797#83;4#Bihar, Bihar, India#IN#IN34#17615#25.1833#85.5167#-2091797#372;4#Bihar, Bihar, India#IN#IN34#17615#25.1833#85.5167#-2091797#537;4#Bihar, Bihar, India#IN#IN34#17615#25.1833#85.5167#-2091797#612;4#Bihar, Bihar, India#IN#IN34#17615#25.1833#85.5167#-2091797#739;4#Bih

In [63]:
from gdeltdoc import GdeltDoc, Filters

# Initialize client
gd = GdeltDoc()

# Define a valid filter
# 'timespan' must be a number followed by one of: "HOURS", "DAYS", "MINUTES"
# and written in uppercase, e.g. "48HOURS" (no space!)
f = Filters(
    keyword="Modi",
    timespan="48h"
)

# Execute search
articles = gd.Search(f)

# Convert to DataFrame
df = gd.DataFrame(articles)

# Show results
print(df.head())

AttributeError: 'GdeltDoc' object has no attribute 'Search'

In [64]:
import requests
import datetime
import time
import json

# --- Configuration ---
QUERY_TERM = "modi"
TIME_WINDOW_HOURS = 72  # Total period to search
SLICE_SIZE_HOURS = 1    # Chunk size for each API request
MAX_RECORDS = 250       # Max records to request per slice (GDELT limit)
PAUSE_SECONDS = 6       # Pause to respect GDELT API rate limit (e.g., >5 seconds)
OUTPUT_FILENAME = f"{QUERY_TERM}_gdelt_data.json"

# Base URL for the GDELT DOC 2.0 API
BASE_URL = "https://api.gdeltproject.org/api/v2/doc/doc"

# Initialize an empty list to hold all collected articles
all_articles = []
successful_requests = 0

# --- Time Calculation ---
# Get current time (T_0h)
end_time = datetime.datetime.now()

# Get the start time (T-72h)
start_time_limit = end_time - datetime.timedelta(hours=TIME_WINDOW_HOURS)

# Start iterating from the oldest time slice
current_start = start_time_limit
print(f"Starting scrape for '{QUERY_TERM}' from {start_time_limit.strftime('%Y-%m-%d %H:%M:%S')} to {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

while current_start < end_time:
    # Define the end of the current slice
    current_end = current_start + datetime.timedelta(hours=SLICE_SIZE_HOURS)
    if current_end > end_time:
        current_end = end_time

    # Format dates to GDELT's required YYYYMMDDHHMMSS format
    # Note: GDELT requires the time to be precise for date filtering
    start_str = current_start.strftime("%Y%m%d%H%M%S")
    end_str = current_end.strftime("%Y%m%d%H%M%S")

    # --- Construct API URL for the current slice ---
    params = {
        'query': QUERY_TERM,
        'startdatetime': start_str,
        'enddatetime': end_str,
        'mode': 'artlist',
        'maxrecords': MAX_RECORDS,
        'format': 'json'
    }

    try:
        # Make the API request
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

        data = response.json()
        
        # GDELT JSON structure contains the articles under the 'articles' key
        if 'articles' in data and data['articles']:
            article_count = len(data['articles'])
            all_articles.extend(data['articles'])
            successful_requests += 1

            print(f"✅ Slice {start_str} to {end_str}: Retrieved {article_count} articles. Total: {len(all_articles)}")
            
            # --- Check if we hit the cap (need to refine slice size) ---
            if article_count >= MAX_RECORDS:
                print(f"⚠️ Warning: Hit MAX_RECORDS ({MAX_RECORDS}) cap. Some data may be missing from this {SLICE_SIZE_HOURS}-hour slice.")

        else:
            print(f"🟨 Slice {start_str} to {end_str}: No articles found.")

    except requests.exceptions.RequestException as e:
        print(f"❌ Error during API call for slice {start_str}: {e}")
        # Consider a longer pause or exiting the loop on a critical error

    # Move to the next time slice
    current_start = current_end

    # Pause to respect the API rate limit (Crucial for the GDELT API)
    if current_start < end_time:
        time.sleep(PAUSE_SECONDS)

# --- Final Output ---
print("\n--- Scraping Complete ---")
print(f"Total successful requests: {successful_requests}")
print(f"Total unique articles collected: {len(all_articles)}")

# Save the collected data to a JSON file
with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
    json.dump(all_articles, f, indent=4)

print(f"Data saved to {OUTPUT_FILENAME}")

Starting scrape for 'modi' from 2025-10-24 21:39:23 to 2025-10-27 21:39:23


KeyboardInterrupt: 

In [6]:
import requests
import json

# --- Configuration ---
GDELT_API_URL = "https://api.gdeltproject.org/api/v2/doc/doc"
OUTPUT_FILENAME = "modi_gdelt_results_final_fix.json"

# --- FIXED ADVANCED QUERY ---
# ALL internal spaces are preserved EXCEPT for the spaces after the colons in filters.
ADVANCED_QUERY_FINAL_FIX = (
    # 1. Main topic and boolean logic (spaces here are fine/necessary)
    '"Narendra Modi" (foreign OR visit OR summit) '
    
    # 2. Metadata filters (CRITICAL: NO SPACES AFTER COLONS)
    'sourcecountry:IN sourcelang:eng tone:>5' 
)

params = {
    'query': ADVANCED_QUERY_FINAL_FIX,
    'timespan': '72h',          # Search the most recent 72 hours
    'mode': 'artlist',          # Return a list of articles
    'maxrecords': 250,          # GDELT's hard limit per call
    'format': 'json'            # Return data in JSON format
}

# --- API Call ---
print(f"Submitting query with fixed parameters.")

try:
    response = requests.get(GDELT_API_URL, params=params)
    response.raise_for_status()
    data = response.json() 

    # --- Process and Output ---
    articles = data.get('articles', [])
    
    if articles:
        print(f"\n✅ SUCCESS! Retrieved {len(articles)} articles (capped at 250).")
        print(f"Sample Article Title: {articles[0].get('title')}")
        
        # Save the full JSON data to a file
        with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4)
        print(f"Full metadata saved to {OUTPUT_FILENAME}")
        
    else:
        print("\n🟨 SUCCESSFUL QUERY, but no articles matched ALL highly specific criteria.")
        print("ACTION: Try removing the 'tone:>5' filter to find more results.")
        
except requests.exceptions.HTTPError as e:
    print(f"\n❌ HTTP Error: The server returned an error (Status Code: {e.response.status_code}).")
    print(f"Check the error message in the response text: {e.response.text[:500]}...")
except json.JSONDecodeError:
    # This should no longer happen, but serves as a final check
    print("\n❌ CRITICAL ERROR: Failed to decode JSON despite fixing the query.")
    print("The API is still returning a non-JSON error. Check GDELT service status.")

Submitting query with fixed parameters.

❌ CRITICAL ERROR: Failed to decode JSON despite fixing the query.
The API is still returning a non-JSON error. Check GDELT service status.


In [7]:
import requests

GDELT_API_URL = 'https://api.gdeltproject.org/api/v2/doc/doc'

# Find exact phrase 'Modi' AND ('France' near 'visit') AND repeat the word 'speech' 3 times
ADVANCED_QUERY_SAFE = (
    # 1. Core Topic (start with Exact Phrase)
    '"Narendra Modi" ' 
    
    # 2. Add one layer of Proximity/Repetition
    'repeat3: Ahemdabad'  
    
    # 3. Add Location/Language (space-separated, NO internal spaces)
    'sourcelang:eng'
)

params_test = {
    'query': '"Hurricane Melissa" near3:"Jamaica"',
    'timespan': '72h',
    'mode': 'artlist',
    'maxrecords': 250,
    'format': 'json'
}

response = requests.get(GDELT_API_URL, params=params_test)


In [10]:
len(response.json()['articles'])

250

In [171]:
BASE_URL = "https://api.gdeltproject.org/api/v2/doc/doc"

# Build the query:
# - "Modi" AND "speech"
# - "Delhi" near Modi or speech (within 5 words)
# - Last 30 days
# - English sources (optional)
query_params = {
    'query': 'Modi speech near5:Delhi',
    'mode': 'artlist',           # List of articles
    'maxrecords': 100,           # Max 250 allowed
    'format': 'json',
    'timespan': '30d',           # Last 30 days
    'sourcelang': 'eng',         # English sources (optional)
    'sort': 'datedesc'           # Newest first
}

# Make the request
response = requests.get(BASE_URL, params=query_params)

In [ ]:
import requests
import json
from datetime import datetime

# Define the GDELT DOC 2.0 API endpoint
BASE_URL = "https://api.gdeltproject.org/api/v2/doc/doc"

# Build the query:
# - "Modi" AND "speech"
# - "Delhi" near Modi or speech (within 5 words)
# - Last 30 days
# - English sources (optional)
query_params = {
    'query': 'Modi (speech OR address) near10:"Delhi OR Red Fort OR Parliament" -rally',
    'mode': 'artlist',           # List of articles
    'maxrecords': 100,           # Max 250 allowed
    'format': 'json',
    'timespan': '30d',           # Last 30 days
    'sourcelang': 'eng',         # English sources (optional)
    'sort': 'datedesc'           # Newest first
}

# Make the request
response = requests.get(BASE_URL, params=query_params)

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    
    print(f"Found {len(data.get('articles', []))} articles on Modi speeches in/near Delhi:\n")
    
    for article in data['articles']:
        title = article.get('title', 'No title')
        url = article.get('url', 'No URL')
        date = article.get('date', 'No date')
        source = article.get('domain', 'Unknown source')
        
        print(f"[{date}] {title}")
        print(f"   Source: {source}")
        print(f"   URL: {url}\n")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [172]:
response.json()['articles']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [86]:
import pandas as pd

pd.DataFrame(response.json())

,articles
0,"{'url': 'https://www.tribuneindia.com/news/india/in-kuala-lumpur-jaishankar-vows-to-deepen-ties-across-asia/', 'url_mobile': 'https://www.tribuneindia.com/news/india/in-kuala-lumpur-jaishankar-vows-to-deepen-ties-across-asia/amp', 'title': 'In Kuala Lumpur , Jaishankar vows to deepen ties across Asia', 'seendate': '20251026T211500Z', 'socialimage': 'https://www.tribuneindia.com/sortd-service/imaginary/v22-01/jpg/large/high?url=dGhldHJpYnVuZS1zb3J0ZC1wcm8tcHJvZC1zb3J0ZC9tZWRpYWE4ZjFkMWQwLWIyYWEtMTFmMC1iMDY0LWVkNWUyYTYzNDljNy5qcGc=', 'domain': 'tribuneindia.com', 'language': 'English', 'sourcecountry': 'India'}"
1,"{'url': 'https://www.thehindu.com/news/national/jaishankar-holds-talks-with-malaysian-pm-regional-counterparts-on-sidelines-of-asean-summit/article70206332.ece', 'url_mobile': 'https://www.thehindu.com/news/national/jaishankar-holds-talks-with-malaysian-pm-regional-counterparts-on-sidelines-of-asean-summit/article70206332.ece/amp/', 'title': 'Jaishankar holds talks with Malaysian PM , regional counterparts on sidelines of ASEAN summit', 'seendate': '20251026T221500Z', 'socialimage': 'https://th-i.thgim.com/public/incoming/3s0dnn/article70206333.ece/alternates/LANDSCAPE_1200/20251026198L.jpg', 'domain': 'thehindu.com', 'language': 'English', 'sourcecountry': 'India'}"
2,"{'url': 'https://www.thehindubusinessline.com/news/jaishankar-meets-asean-leaders-discusses-ties-with-malaysia-south-korea-singapore-and-thailand/article70207030.ece', 'url_mobile': 'https://www.thehindubusinessline.com/news/jaishankar-meets-asean-leaders-discusses-ties-with-malaysia-south-korea-singapore-and-thailand/article70207030.ece/amp/', 'title': 'Jaishankar meets ASEAN leaders to boost regional ties - The HinduBusinessLine', 'seendate': '20251027T040000Z', 'socialimage': 'https://bl-i.thgim.com/public/incoming/aycdjc/article70207040.ece/alternates/LANDSCAPE_1200/20251026198L.jpg', 'domain': 'thehindubusinessline.com', 'language': 'English', 'sourcecountry': 'India'}"
3,"{'url': 'https://www.freepressjournal.in/world/47th-asean-summit-eam-s-jaishankar-meets-malaysian-pm-anwar-ibrahim-conveys-pm-modis-wishes', 'url_mobile': 'https://www.freepressjournal.in/amp/world/47th-asean-summit-eam-s-jaishankar-meets-malaysian-pm-anwar-ibrahim-conveys-pm-modis-wishes', 'title': '47th ASEAN Summit : EAM S Jaishankar Meets Malaysian PM Anwar Ibrahim , Conveys PM Modi Wishes', 'seendate': '20251026T184500Z', 'socialimage': 'https://media.assettype.com/freepressjournal/2025-10-26/22900f1p/G4MDvF6XIAEgQUu.jpg', 'domain': 'freepressjournal.in', 'language': 'English', 'sourcecountry': 'India'}"
4,"{'url': 'https://www.livemint.com/news/india/21st-century-belongs-to-india-and-asean-what-modi-said-in-virtual-address-to-22nd-asean-india-summit-check-here-11761478212588.html', 'url_mobile': 'https://www.livemint.com/news/india/21st-century-belongs-to-india-and-asean-what-modi-said-in-virtual-address-to-22nd-asean-india-summit-check-here/amp-11761478212588.html', 'title': ' 21st century belongs to … : What PM Narendra Modi said in his virtual address to 22nd ASEAN - India Summit – Check here', 'seendate': '20251026T130000Z', 'socialimage': 'https://www.livemint.com/lm-img/img/2025/10/26/600x338/PTI10-26-2025-000150B-0_1761478331711_1761478412334.jpg', 'domain': 'livemint.com', 'language': 'English', 'sourcecountry': 'India'}"
...,...
245,"{'url': 'http://www.calcuttanews.net/news/278657310/seychelles-vital-in-advancing-mahasagar-vision-priorities-of-global-south-vp-radhakrishnan', 'url_mobile': '', 'title': 'Seychelles vital in advancing MAHASAGAR vision , priorities of Global South : VP Radhakrishnan', 'seendate': '20251026T191500Z', 'socialimage': 'https://image.chitra.live/api/v1/wps/87c3bee/fdef4a93-6112-4b1b-9ba0-b136b223c4e6/0/ANI-20251026171245-600x315.jpg', 'domain': 'calcuttanews.net', 'language': 'English', 'sourcecountry': 'India'}"
246,"{'url': 'https://www.firstpost.com/explainers/donald-trump-asia-tour-world-leaders-jet-lag-ws-e-13945444.html

In [29]:
import requests
from datetime import datetime, timedelta
import json
import csv
def build_query() -> str:
    # NOTE: the whole near‑phrase is inside ONE quoted string
    return 'Modi (speech OR address) near10:"Delhi OR Red Fort OR Parliament" -rally'

# -------------------------------------------------
# 2. Time window (last 30 days – adjust as you like)
# -------------------------------------------------
END   = datetime(2025, 10, 28)                     # today (or datetime.utcnow())
START = END - timedelta(days=30)

# -------------------------------------------------
# 3. Base parameters (requests will URL‑encode everything)
# -------------------------------------------------
BASE_URL = "https://api.gdeltproject.org/api/v2/doc/doc"

params = {
    "query":          result.split(':')[1].strip(" "),
    "mode":           "artlist",          # list of articles
    "maxrecords":     100,                # 250 is the absolute max
    "format":         "json",
    'timespan':       '72h',           # Last 30 days
    "sourcelang":     "eng",              # remove for multilingual
    "sort":           "datedesc"          # newest first
}

# -------------------------------------------------
# 4. Make the request
# -------------------------------------------------
print(f"Querying GDELT …")
resp = requests.get(BASE_URL, params=params, timeout=30)

# -------------------------------------------------
# 5. Handle response
# -------------------------------------------------
if resp.status_code != 200:
    print(f"API error {resp.status_code}")
    print(resp.text[:500])
    raise SystemExit



Querying GDELT …


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [53]:
import requests
from datetime import datetime, timedelta
from langchain_ollama.llms import OllamaLLM


# 1. Local Ollama model
llm = OllamaLLM(model="deepseek-r1:8b", temperature=0.1)

# 2. Simple prompt (few examples = reliable output)
prompt = """You are a GDELT DOC 2.0 query generator.  
Return **exactly one line**:

QUERY: <valid-gdelt-query>

Rules (follow 100%):
- Exact phrase → "Red Fort"
- AND → space
- OR → (A OR B) - **uppercase OR**
- NOT → -word
- Proximity → near5:"Delhi" **(quotes REQUIRED)**
- No time, mode, format, or parentheses around a single phrase

Examples (copy the pattern exactly):
1. Trump speeches in Washington, no rallies → QUERY: Trump (speech OR address) near10:"Washington" -rally
2. COVID vaccines but not China → QUERY: covid vaccine -china
3. Modi assassination attempt that mentions China → QUERY: modi "assassination attempt" china
4. Protests near Eiffel Tower in French → QUERY: protest near5:"Eiffel Tower" sourcelang:fra

User: {input}
"""

# 3. Generate query
def make_gdelt_query(text: str) -> str:
    full = prompt.format(input=text)
    result = llm.invoke(full)
    # Extract text after "Query:'"
    if "Query:'" in result:
        query = result.split("Query:'")[1].strip().strip("'\"")
    else:
        query = result.strip().strip("'\"")
    return query

In [58]:
full = prompt.format(input="Oracle deal with Open AI")
result = llm.invoke(full)

In [59]:
result

'QUERY: Oracle deal with Open AI'

In [4]:
import requests

url = "https://api.gdeltproject.org/api/v2/doc/doc"
params = {
    "query": 'modi "assasination attempt" near5:"China"',  # Now includes China
    "mode": "artlist",
    "maxrecords": 100,
    "format": "json",
    "timespan": "1yr",
    "sourcelang": "eng",
    "sort": "datedesc"
}

response = requests.get(url, params=params)
data = response.json()
articles = data.get("articles", [])

In [5]:
len(articles)

0

In [20]:
full = prompt.format(input="modi assasination attempt in china")
result = llm.invoke(full)

In [51]:
result

'QUERY: modi assasination attempt near1:china'

In [10]:
response=make_gdelt_query("modi assasination attempt in china")

KeyboardInterrupt: 

In [6]:
import pkgutil
import importlib
import langchain_ollama

print("LangChain Ollama version:", langchain_ollama.__version__)
print("\nAvailable submodules:")
for module in pkgutil.iter_modules(langchain_ollama.__path__):
    print(f"- {module.name}")

# Try to import Ollama directly
try:
    from langchain_ollama import Ollama
    print("\nSuccessfully imported Ollama!")
except ImportError as e:
    print(f"\nError importing Ollama: {e}")

LangChain Ollama version: 1.0.0

Available submodules:
- _compat
- _utils
- chat_models
- embeddings
- llms

Error importing Ollama: cannot import name 'Ollama' from 'langchain_ollama' (/Users/gurusai/programming/PERSONAL_NEWS/.venv/lib/python3.13/site-packages/langchain_ollama/__init__.py)


In [1]:
from Crawler import GdeltsCrawler

crawler = GdeltsCrawler()

dickt = {'query': 'Hurricane Melissa near3:"Jamaica"'}

articles = crawler.retrieve(**dickt)


In [2]:
articles[articles['language']=='English']

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,https://www.jamaicaobserver.com/2025/10/30/melissaaftermath-france-dispatching-humanitarian-aid-hurricane-wrecked-jamaica/,,# MelissaAftermath : France dispatching humanitarian aid to hurricane - wrecked Jamaica,20251030T160000Z,https://www.jamaicaobserver.com/jamaicaobserver/news/wp-content/uploads/sites/4/2025/10/WhatsApp-Image-2025-10-30-at-10.31.29-AM.jpeg,jamaicaobserver.com,English,Jamaica
1,https://www.yahoo.com/news/articles/hurricane-melissa-rampages-across-jamaica-121735131.html,,"Hurricane Melissa rampages across Jamaica , Cuba , now Cat 1 . Here where it going next",20251030T160000Z,https://media.zenfs.com/en/the-news-press/2fafd0bceba4ac149340a9816340d280,yahoo.com,English,United States
2,https://720thevoice.iheart.com/content/2025-10-30-hurricane-melissa-devastation-on-level-never-seen-before-un/,,Hurricane Melissa Devastation On Level Never Seen Before : UN,20251030T160000Z,"https://i.iheart.com/v3/re/new_assets/690245e43b78354d01b1ccf7?ops=gravity(%22north%22),fit(1200,675),quality(65)",720thevoice.iheart.com,English,United States
3,https://wtkg.iheart.com/content/2025-10-30-hurricane-melissa-devastation-on-level-never-seen-before-un/,,Hurricane Melissa Devastation On Level Never Seen Before : UN,20251030T160000Z,"https://i.iheart.com/v3/re/new_assets/690245e43b78354d01b1ccf7?ops=gravity(%22north%22),fit(1200,675),quality(65)",wtkg.iheart.com,English,United States
4,https://www.cleveland19.com/2025/10/30/haiti-jamaica-cuba-pick-up-pieces-after-melissas-destruction/,https://www.cleveland19.com/2025/10/30/haiti-jamaica-cuba-pick-up-pieces-after-melissas-destruction/?outputType=amp,"Haiti , Jamaica and Cuba pick up the pieces after Melissa destruction",20251030T160000Z,https://gray-woio-prod.gtv-cdn.com/resizer/v2/BVUC6LRDS5HOTKTZPPXWMVS3CQ.jpg?auth=fd3d36037b23ade7d6cb5bd9b80b9518def4df16be3abcd4900ad2e08e252a5f&smart=true,cleveland19.com,English,United States
...,...,...,...,...,...,...,...,...
245,https://www.runcornandwidnesworld.co.uk/news/national/25583046.hurricane-melissa-shows-need-climate-action-cop30-summit-cabinet-hears/,,"Hurricane Melissa shows need for climate action at Cop30 summit , Cabinet hears",20251030T120000Z,https://www.runcornandwidnesworld.co.uk/resources/images/20138289.jpg?type=og-image,runcornandwidnesworld.co.uk,English,United Kingdom
246,https://www.kilburntimes.co.uk/news/national/25583183.haiti-jamaica-cuba-pick-pieces-melissas-destruction/,,"Haiti , Jamaica and Cuba pick up the pieces after Melissa destruction",20251030T120000Z,https://www.kilburntimes.co.uk/resources/images/20138359.jpg?type=og-image,kilburntimes.co.uk,English,United Kingdom
247,https://www.bournemouthecho.co.uk/news/national/25583183.haiti-jamaica-cuba-pick-pieces-melissas-destruction/,,"Haiti , Jamaica and Cuba pick up the pieces after Melissa destruction",20251030T120000Z,https://www.bournemouthecho.co.uk/resources/images/20138359.jpg?type=og-image,bournemouthecho.co.uk,English,United Kingdom
248,https://www.sloughobserver.co.uk/news/national/25583183.haiti-jamaica-cuba-pick-pieces-melissas-destruction/,,"Haiti , Jamaica and Cuba pick up the pieces after Melissa destruction",20251030T120000Z,https://www.sloughobserver.co.uk/resources/images/20138359.jpg?type=og-image,sloughobserver.co.uk,English,United Kingdom


In [7]:
testUrl = 'https://www.jamaicaobserver.com/2025/10/30/melissaaftermath-france-dispatching-humanitarian-aid-hurricane-wrecked-jamaica/'

from newspaper import Article
import nltk
nltk.download('punkt_tab')
article = Article(testUrl)
article.download()
article.parse()
article.nlp()
print(article.text)
output = {
            'title': article.title,
            'text': article.text,
            'publish_date': article.publish_date,
            'authors': article.authors,
            'top_image': article.top_image,
            'summary': article.summary
        }

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/gurusai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Latest News, News

MANCHESTER, Jamaica — Scores of residents converged at the Mid-Island Service Station at the border of Manchester and St Elizabeth on Thursday, clutch...

{"jamaica-observer":"Jamaica Observer"}

Latest News, News

KINGSTON, Jamaica — Popular food joint Chicken and Tings has teamed up with international non-profit World Central Kitchen (WCK) to provide thousands ...

{"jamaica-observer":"Jamaica Observer"}

Latest News

KINGSTON, Jamaica — As Jamaica continues to recover from the devastating impacts of Hurricane Melissa, Optimum Jamaica is offering free calls and text...

{"jamaica-observer":"Jamaica Observer"}

Latest News

KINGSTON, Jamaica — International Christian relief organisation, Samaritan’s Purse, says its disaster response teams and supplies are on standby to be...

{"jamaica-observer":"Jamaica Observer"}

Latest News, News

Grierfield resident recounts Hurricane Melissa horror

SANDENA JAMES, Social Media Editor

In the rural community of Grierfield, Mone

In [18]:
import trafilatura
testUrl = 'https://www.jamaicaobserver.com/2025/10/30/melissaaftermath-france-dispatching-humanitarian-aid-hurricane-wrecked-jamaica/'
downloaded = trafilatura.fetch_url(articles['url'].iloc[2])
text = trafilatura.extract(downloaded, include_comments=False, 
                            include_tables=True)
print(text)

Recovery efforts are underway after Hurricane Melissa left a path of devastation in the Caribbean this week.
The United Nations said the damage in Jamaica, where the storm made landfall on Tuesday (October 28) as a Category 5 hurricane, was on a level "never seen before." Cuba is also reported to be calculating cost of damages after homes collapsed and blocked roads, with an estimated 735,000 people reported to be in shelters and the full extent of damage undetermined.
At least 31 people have died in relation to Hurricane Melissa's devastation across several countries. At least 25 people died and several remain trapped in homes in Petit-Goáve, Haiti, after a river was flooded by the powerful storm, Mayor Jean Bertrans Subrème told the Associated Press.
“I am overwhelmed by the situation,” Subrème said, adding that he’d requested assistance from the government.
At least three other deaths, including two caused by a landslide, were also reported in Haiti in relation to Hurricane Melissa,

In [14]:
articles['url'].iloc[0]

'https://www.jamaicaobserver.com/2025/10/30/melissaaftermath-france-dispatching-humanitarian-aid-hurricane-wrecked-jamaica/'

In [9]:
output['summary']

'Latest News, News MANCHESTER, Jamaica — Scores of residents converged at the Mid-Island Service Station at the border of Manchester and St Elizabeth on Thursday, clutch... {"jamaica-observer":"Jamaica Observer"} Latest News, News KINGSTON, Jamaica — Popular food joint Chicken and Tings has teamed up with international non-profit World Central Kitchen (WCK) to provide thousands ... {"jamaica-observer":"Jamaica Observer"} Latest News KINGSTON, Jamaica — As Jamaica continues to recover from the devastating impacts of Hurricane Melissa, Optimum Jamaica is offering free calls and text... {"jamaica-observer":"Jamaica Observer"} Latest News KINGSTON, Jamaica — International Christian relief organisation, Samaritan’s Purse, says its disaster response teams and supplies are on standby to be... {"jamaica-observer":"Jamaica Observer"} Latest News, News Grierfield resident recounts Hurricane Melissa horror SANDENA JAMES, Social Media Editor In the rural community of Grierfield, Moneague, St Ann, 

In [4]:
output

{'title': '#MelissaAftermath: France dispatching humanitarian aid to hurricane',
 'text': 'Latest News, News\n\nMANCHESTER, Jamaica — Scores of residents converged at the Mid-Island Service Station at the border of Manchester and St Elizabeth on Thursday, clutch...\n\n{"jamaica-observer":"Jamaica Observer"}\n\nLatest News, News\n\nKINGSTON, Jamaica — Popular food joint Chicken and Tings has teamed up with international non-profit World Central Kitchen (WCK) to provide thousands ...\n\n{"jamaica-observer":"Jamaica Observer"}\n\nLatest News\n\nKINGSTON, Jamaica — As Jamaica continues to recover from the devastating impacts of Hurricane Melissa, Optimum Jamaica is offering free calls and text...\n\n{"jamaica-observer":"Jamaica Observer"}\n\nLatest News\n\nKINGSTON, Jamaica — International Christian relief organisation, Samaritan’s Purse, says its disaster response teams and supplies are on standby to be...\n\n{"jamaica-observer":"Jamaica Observer"}\n\nLatest News, News\n\nGrierfield resid

In [10]:
%pip install trafilatura

  Using cached lxml-5.4.0-cp313-cp313-macosx_10_13_universal2.whl.metadata (3.5 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 40.6 MB/s  0:00:00eta 0:00:01
Using cached lxml-5.4.0-cp313-cp313-macosx_10_13_universal2.whl (8.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 19.9 MB/s  0:00:00
Using cached tzlocal-5.3.1-py3-none-any.whl (18 kB)
  Attempting uninstall: lxml
    Found existing installation: lxml 6.0.2
    Uninstalling lxml-6.0.2:
      Successfully uninstalled lxml-6.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [trafilatura] [dateparser]
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install sumy

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def summarize_text(text, sentences_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentences_count)
    return " ".join([str(sentence) for sentence in summary])

# Then use it like:
clean_text = text
if clean_text:
    summary = summarize_text(clean_text)
    print("Summary:", summary)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.4 MB/s  0:00:00
Using cached chardet-5.2.0-py3-none-any.whl (199 kB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13783 sha256=9dd5467c846586480f9ddf60c425498c1bcc8fd978b6c0ca8e6487cfcc9c08ba
  Stored in directory: /Users/gurusai/Library/Caches/pip/wheels/0b/1d/03/175286677fb5a1341cc3e4755bf8ec0ed08f3329afd67446b0
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21794 sha256=0bf8213c0679f13ca9efb45f12403f6e8f61e32810c1602ac6e6cdafede62dd1
  Stored in directory: /Users/gurusai/Library/Caches/pip/wheels/56/de/fa/27aad3b8cf9dc6bf1675f

In [20]:
summary

'Recovery efforts are underway after Hurricane Melissa left a path of devastation in the Caribbean this week. At least one person has died in the Dominican Republic, according to officials, who confirmed more than 1,000 others were evacuated or displaced via CNN. Jamaica was reported to have "suffered major impact" after the hurricane made landfall, with at least two or three hospitals suffering severe damage and housing expected to be "severely impacted" in the storm\'s path, Prime Minister Andrew Holness said via NBC News.'